In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import json
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.api import STLForecast
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
from math import sin, cos, sqrt, atan2, radians
from tqdm import tqdm

np.seterr(invalid='ignore')

In [ ]:
!wget -nc https://archive.ics.uci.edu/static/public/501/beijing+multi+site+air+quality+data.zip -O ./data/data.zip

os.makedirs('./data/PRSA_Data', exist_ok=True)

!unzip -n ./data/data.zip -d ./data
!unzip -n ./data/PRSA2017_Data_20130301-20170228.zip -d data/
!mv --no-clobber ./data/PRSA_Data_20130301-20170228/* ./data/PRSA_Data

### Setup Functions

In [ ]:
def process_data(path: str, encode_wd: bool=False) -> pd.DataFrame:
    df = pd.read_csv(path, parse_dates={'timestamp': ['year', 'month', 'day', 'hour']}, date_format="%Y %m %d %H")
    df = df.set_index('timestamp')

    if encode_wd:
        one_hot = pd.get_dummies(df['wd'].fillna('NAN'), dtype=int)
        one_hot.loc[one_hot['NAN'] == 1, list(one_hot.columns)] = np.nan
        one_hot = one_hot.drop(['NAN'], axis=1)
        df = df.join(one_hot)
        df = df.drop(['wd'], axis=1)
        
    df = df.interpolate(method='time')
    df = df.interpolate(method='backfill')
    df = df.drop(['No'], axis=1)

    return df

def visualize_forecasts(full_series: pd.Series, base_forecast: pd.Series, rmse_base: float, triangulation_forecast: pd.Series, rmse_dis:float, column:str, station:str) -> None:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=full_series.index, y=full_series.values, mode='lines', name='Complete Series'))
    fig.add_trace(go.Scatter(x=base_forecast.index, y=base_forecast.values, mode='lines', name='Simple Forecast'))
    fig.add_trace(go.Scatter(x=triangulation_forecast.index, y=triangulation_forecast.values, mode='lines', name='Triangulation Forecast'))

    annotations = []
    annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1, xanchor='center', yanchor='top', showarrow=False,
                            text=f'Simple Forecast RMSE:   {round(rmse_base, 4)}', font=dict(family='Arial', size=14)))
    annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.2, xanchor='center', yanchor='top', showarrow=False,
                            text=f'Triangulation Forecast RMSE:  {round(rmse_dis, 4)}', font=dict(family='Arial', size=14)))
    
    
    fig.update_layout(
        title=dict(
            text=f'Forecastings of {column} over time in {station} Air Station (Beijing)'
        ),
        yaxis=dict(
            title=dict(
                text=f'{column}'
            )
        ),
        annotations=annotations
    )

    fig.show()

### Goes through 27 combinations of parameters in a simple ARIMA to find the best values
def get_best_params(series:pd.Series, division:int, param_grid:dict={'p': [0,1, 2], 'd': [0,1, 2], 'q': [0,1,2 ]}) -> dict:
    grid = ParameterGrid(param_grid)
    
    train_series = series[:-division]
    train_series.index = pd.DatetimeIndex(train_series.index.values, freq=train_series.index.inferred_freq)
    test_series = series[-division:]

    rmse = []

    for params in tqdm(grid, total=len(grid)):
        model = ARIMA(train_series, order=(params['p'], params['d'], params['q']))
        model_fit = model.fit()

        pred = model_fit.get_forecast(len(test_series.index))
        pred_series = pred.conf_int(alpha = 0.05) 
        pred_series["Predictions"] = model_fit.predict(start = pred_series.index[0], end = pred_series.index[-1])
        pred_series.index = test_series.index
        arima_rmse = np.sqrt(mean_squared_error(test_series.values, pred_series["Predictions"]))

        rmse.append(arima_rmse)

    tuning_results = pd.DataFrame(grid)
    tuning_results['rmse'] = rmse
    best_params = tuning_results[tuning_results.rmse == tuning_results.rmse.min()].transpose()

    return list(best_params.to_dict().values())[0]

### Distance Based Functions

In [ ]:
coordinate_dict = {'Huairou': (116.17, 40.09),
'Changping': (116.21, 40.00),
'Wanliu': (116.29, 39.98),
'Aotizhongxin': (116.40, 39.98),
'Gucheng': (116.18, 39.91),
'Shunyi': (116.14, 39.82),
'Guanyuan': (116.34, 39.92),
'Dongsi': (116.42, 39.93),
'Tiantan': (116.41, 39.88),
'Nongzhanguan': (116.47, 39.93),
'Dingling': (116.28, 39.86),
'Wanshouxigong': (116.35, 39.88)}

def distance_km(coord1: tuple[float, float], coord2: tuple[float, float]) -> float:
    R = 6373.0

    lat1 = radians(coord1[0]) 
    lon1 = radians(coord1[1])
    lat2 = radians(coord2[0])
    lon2 = radians(coord2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

def get_weights_to_centerpoint(coordinate_dict:dict, centerpoint: str):
    distances_dict = {}

    for key in coordinate_dict.keys(): ## Loop gets the distances from all other stations to a given centerpoint
        if key != centerpoint:
            distances_dict[key] = distance_km(coordinate_dict[centerpoint], coordinate_dict[key])

    total = sum(distances_dict.values())

    weights_dict = {}
    for key in distances_dict.keys(): ## Loop calculates a weight inversely proportional to the distance to the centerpoint
        weights_dict[key] = total / distances_dict[key]

    total_weight = sum(weights_dict.values())
    for key in weights_dict.keys(): ## Loop normalizes the weights
        weights_dict[key] = weights_dict[key]/total_weight

    return weights_dict   
    

def get_triangulation_forecast(series_dict: dict[pd.Series], division: int, best_params: dict, periodicity:int, coordinate_dict:dict, centerpoint: str):

    centerpoint_series = series_dict[centerpoint]
    test = centerpoint_series[-division:]

    forecasts = {}
    for key in tqdm(series_dict.keys(), total=len(series_dict.keys())): ## Loop gets the forecasting predicted for each non-centerpoint station
        if key != centerpoint:
            train = series_dict[key][:-division]

            ### Uses STL to apply seasonality on ARIMA
            model = STLForecast(train, ARIMA, model_kwargs={'order':(best_params['p'], best_params['d'], best_params['q'])}, period=periodicity)
            model_fit = model.fit()

            pred = model_fit.forecast(len(test.index))
            pred = pd.Series(pred)
            pred.index = test.index


            forecasts[key] = np.array(pred)

    weights = get_weights_to_centerpoint(coordinate_dict=coordinate_dict, centerpoint=centerpoint)

    for key in forecasts.keys(): ## Loop applies the weights to all forecastings
        forecasts[key] = forecasts[key] * weights[key]

    forecast_matrix = np.array(list(forecasts.values()))
    final_forecast = forecast_matrix.sum(axis=0)
    
    arima_rmse = np.sqrt(mean_squared_error(test.values, final_forecast))
    
    return pd.Series(final_forecast, index=test.index), arima_rmse


### Running

In [ ]:
path = './data/PRSA_Data'
file_list = os.listdir(path)

### Amount of hours in other scales
day = 24
month = 720
year = 8760

In [ ]:
### This cell finds and stores the best out 27 possible ARIMA parameter combinations for each univariate series in the Dataset

if 'bestParams.json' in os.listdir('data/'):
    with open('data/bestParams.json', 'r') as f: best_params_dict = json.load(f)
else:
    df = process_data(f'{path}/{file_list[0]}')
    best_params_dict = {}
    for column in tqdm(df.columns, total=len(df.columns)):
        if column not in ['wd', 'station']:
            series = df[column]
            best_params = get_best_params(series, division=3*month)
            best_params_dict[column] = best_params

    with open("data/bestParams.json", 'w') as f: json.dump(best_params_dict, f)

In [ ]:
best_params_dict.keys() # List the series

In [ ]:
### Control cell for the bellow one
# column --------> Defines the univariate series to be explored
# centerpoint ---> Defines the Station whose time series will be forecast using both simple forecasting and the novel triangulation forecasting
# best_params ---> Gets the best ARIMA parameters for a given univariate series
# division ------> Defines the train/test split for the forecast, for example <<year + 3*month>> means that the last 3 months and one year will be forecast and the rest will be used for training]
# peridiocity ---> Hyperparameter that defines the period of a series, for example a value of <<year>> means yearly periodicity. Can be set to <<None>>
column = 'TEMP' 
centerpoint = 'Wanliu'
best_params = best_params_dict[column]
division = year + 3*month
periodicity = year

In [ ]:
### According to the parameters on the cell above calculates simple and triangulation forecast of a given Centerpoint

centerpoint_file = [file for file in file_list if centerpoint in file][0]
full_series = process_data(f'{path}/{centerpoint_file}')[column]

train = full_series[:-division]
test = full_series[-division:]

### Uses STL to apply seasonality on ARIMA
model = STLForecast(train, ARIMA, model_kwargs={'order':(best_params['p'], best_params['d'], best_params['q'])}, period=periodicity)
model_fit = model.fit()

base_forecast = model_fit.forecast(len(test.index))
base_forecast = pd.Series(base_forecast)
base_forecast.index = test.index
rmse_base = np.sqrt(mean_squared_error(test.values, base_forecast))

series_dict = {}
for file in file_list:
    df = process_data(f'{path}/{file}')
    station_name = file.split('_')[2]
    series_dict[station_name] = df[column]

triangulation_forecast, rmse_dis = get_triangulation_forecast(series_dict, division, best_params, periodicity, coordinate_dict, centerpoint=centerpoint)

In [ ]:
visualize_forecasts(full_series, base_forecast=base_forecast, rmse_base=rmse_base, triangulation_forecast=triangulation_forecast, rmse_dis=rmse_dis, column=column, station=centerpoint)